# Image Classification of Documents

### 1.0 Import packages and libraries
Import the packages and libraries that you'll use:

In [1]:
import os, random
import numpy as np
import pandas as pd
import PIL
import keras
import itertools
from PIL import Image

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
# from skimage import feature, data, io, measure
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
from matplotlib import ticker
import seaborn as sns
%matplotlib inline 

from keras import backend as K
from keras.models import Sequential
from keras.layers import Input, Dropout, Flatten, Conv2D, MaxPooling2D, Dense, Activation
from keras.optimizers import RMSprop, Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


Using TensorFlow backend.


### 2 Global Variables 
Enter the batch size for training, testing and validation dataset

In [2]:
batch_size_train = 20
batch_size_val = 10
batch_size_test = 25
num_classes= 3
intereseted_folder='Documents'
STANDARD_SIZE=(224,224)

# 3. Classification

## 3.1 Create the Datset

In [3]:
#Converting Data Format according to the backend used by Keras

classes_required = ['Cat', 'Driving_License', 'Pancard']

In [4]:
datagen=keras.preprocessing.image.ImageDataGenerator(data_format=K.image_data_format())

In [5]:
#Input the Training Data

train_path = r'C:\Users\Archit\Desktop\CV_AA\CV2\catVsdog\training_set'
train_batches = ImageDataGenerator().flow_from_directory(train_path, target_size=(224,224),  batch_size=batch_size_train)
type(train_batches)

Found 2000 images belonging to 2 classes.


keras.preprocessing.image.DirectoryIterator

In [6]:
#Input the Validation Data
val_path = r'C:\Users\Archit\Desktop\CV_AA\CV2\catVsdog\test_set'
val_batches = ImageDataGenerator().flow_from_directory(val_path, target_size=(224,224), batch_size=batch_size_val)
type(val_batches)

Found 1000 images belonging to 2 classes.


keras.preprocessing.image.DirectoryIterator

In [7]:
# #Input the Test Data
# test_path = r'C:\Users\Archit\Desktop\CV_AA\CV2\Images\Test'
# test_batches = ImageDataGenerator().flow_from_directory(test_path, target_size=(224,224), classes=classes_required, batch_size=batch_size_test)
# type(test_batches)

In [8]:
# next(test_batches)

In [9]:
train_imgs, train_labels = next(train_batches)


In [10]:
# train_imgs

In [11]:
# train_labels

In [12]:
# test_imgs, test_labels = next(test_batches)

In [13]:
# test_imgs

In [14]:
# test_labels

In [15]:
# y_test= [ np.where(r==1)[0][0] for r in test_labels ]
# y_test

## 3.2 Build the Model

In [16]:
model_ince = keras.applications.InceptionV3()

In [17]:
model_ince.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 149, 149, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 149, 149, 32) 0           batch_normalization_1[0][0]      
_______________________________________________________________________________________

In [18]:
model_ince.layers

In [19]:
vgg16_model = keras.applications.vgg16.VGG16()
vgg16_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [20]:
type(vgg16_model) #This is a Keras Functional API need to convert to sequential

keras.engine.training.Model

In [21]:
vgg16_model.layers

In [22]:
model = Sequential() #Iterate over the functional layers and add it as a stack
for layer in vgg16_model.layers:
    model.add(layer)

In [23]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [24]:
# last=model.get

In [25]:
for layer in model.layers: #Since the model is already trained with certain weights, we dont want to change it. Let it be the same
    layer.trainable = False

In [26]:
model.add(Dense(2, activation='softmax')) # Add the last layer

In [27]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [28]:
# Complie the model
model.compile(Adam(lr=.00015), loss='binary_crossentropy', metrics=['accuracy'])

## 3.3 Train the Model

The model will take about 30-45 minutes to train. 

In [30]:
model.fit_generator(train_batches, steps_per_epoch=20, 
                    validation_data=val_batches, validation_steps=20, epochs=10, verbose=1)

Epoch 1/10
20/20 [==============================] - 89s 4s/step - loss: 0.6882 - accuracy: 0.6000 - val_loss: 0.6850 - val_accuracy: 0.6150
Epoch 2/10
20/20 [==============================] - 93s 5s/step - loss: 0.6872 - accuracy: 0.6300 - val_loss: 0.6763 - val_accuracy: 0.5900
Epoch 3/10
20/20 [==============================] - 97s 5s/step - loss: 0.6841 - accuracy: 0.6625 - val_loss: 0.6907 - val_accuracy: 0.6800
Epoch 4/10
20/20 [==============================] - 100s 5s/step - loss: 0.6846 - accuracy: 0.6625 - val_loss: 0.6890 - val_accuracy: 0.7750
Epoch 5/10
20/20 [==============================] - 102s 5s/step - loss: 0.6816 - accuracy: 0.7300 - val_loss: 0.6749 - val_accuracy: 0.7500
Epoch 6/10
20/20 [==============================] - 103s 5s/step - loss: 0.6801 - accuracy: 0.7525 - val_loss: 0.6710 - val_accuracy: 0.7950
Epoch 7/10
20/20 [==============================] - 105s 5s/step - loss: 0.6809 - accuracy: 0.7650 - val_loss: 0.6845 - val_accuracy: 0.7350
Epoch 8/10
20/20

In [ ]:
%pwd

In [ ]:
model.save_weights('my_model_weights.h5')
model.load_weights('my_model_weights.h5')

In [ ]:
model.summary()

In [ ]:
# model.get_weights()

## 3.4 Test the Model with External Test Images

In [ ]:
zip_ref = r'C:\Users\Archit\Desktop\CV_AA\CV2\Images'

In [ ]:
paths = [zip_ref+"/gh.jpg", zip_ref+"/th.jpg",zip_ref+"/th1.jpg",zip_ref+"/th2.jpg",zip_ref+"/th3.jpg"]

In [ ]:
paths

In [ ]:
X_test=[]
def convert_to_image(X):
    '''Function to convert all Input Images to the STANDARD_SIZE and create Training Dataset
    '''
    for f in paths:
        #fobj=get_file(f)
        #print(type(fobj))predictions= model.predict(X_test)
        if os.path.isdir(f):
            continue
        img= PIL.Image.open(f)
        img = img.resize(STANDARD_SIZE)
        img=np.array(img)
        X.append(img)
        #print(X_train)
    #print(len(X_train))
    return X
X_test=np.array(convert_to_image(X_test))
datagen.fit(X_test)

In [ ]:
X_test

In [ ]:
predictions= model.predict(X_test)
predictions

In [ ]:
y_pred=[]
for i in range(len(predictions)):
    y_pred.append(np.argmax(predictions[i]))

In [ ]:
y_pred

In [ ]:
classes_required = ['Cheque', 'Driving_License', 'Pancard']